In [1]:
# import deepmatcher as dm
import itertools
import os
import pandas as pd
# import model_discovery
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

# from model_discovery import utils, l2lsh
import re

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet


import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras 
from keras import backend as K
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM,Dense,Bidirectional,Input
from keras.models import Model
import torch
import transformers

In [2]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

def fast_encode(texts, tokenizer_fuc, chunk_size=256, maxlen=400):

    tokenizer_fuc.enable_truncation(max_length=maxlen)
    tokenizer_fuc.enable_padding(length=maxlen)
    all_ids = []
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer_fuc.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)


In [3]:
def build_model(transformer, max_len=400):
    
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

from tokenizers import BertWordPieceTokenizer
# First load the real tokenizer
tokenizer_ = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased' , lower = True)
# Save the loaded tokenizer locally
tokenizer_.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
bert_model = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

2022-04-13 17:19:31.911840: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-13 17:19:31.913018: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-13 17:19:31.913684: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-13 17:19:31.914534: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
result_df=pd.read_csv('../input/ver-1-of-model-search-dataset/Sentiment_serving-train.csv')
result_df

,Source,Target,JSD,Adaptivity,L2D,Source Accuracy,Serving Accuracy,Finetune Accuracy
0,Financial_News_1,Tweets_hatred_5,0.508087,1.266667,0.153876,87.804878,76.717216,71.275645
1,Financial_News_2,Tweets_hatred_5,0.499096,1.466667,0.153196,88.617885,72.227180,76.984835
2,Movie_2,Tweets_hatred_5,0.114503,15.800000,0.073590,86.879998,66.577458,59.500444
3,Tweets_2,Tweets_hatred_5,0.093903,17.000000,0.052303,71.759999,64.644665,59.322035
4,Movie_1,Tweets_hatred_5,0.113434,16.133333,0.073737,84.320003,59.024680,74.041033
5,Tweets_1,Tweets_hatred_5,0.097748,17.000000,0.052496,68.320000,57.567650,54.326493
6,Tweets_1,Tweets_2,0.006653,17.000000,0.013643,68.320000,70.933336,72.479999
7,Financial_News_1,Tweets_2,0.530934,1.176471,0.170825,87.804878,64.240003,70.160002
8,Financial_News_2,Tweets_2,0.523461,1.352941,0.167609,88.617885,63.546669,68.720001
9,Movie_2,Tweets_2,0.123531,15.470588,0.079556,86.879998,63.226664,69.840002


In [5]:
datasets = ['../input/imbd-data-1/Movie_1.csv', '../input/imbd-data-1/Movie_2.csv', 
            '../input/financial-news-data-2/Financial_News_2.csv', '../input/financial-news-data-2/Financial_News_1.csv',
            '../input/tweets-data-4/Tweets_1.csv','../input/tweets-data-4/Tweets_2.csv',
           '../input/tweets-hatred-data-5/Tweets_hatred_5.csv']

In [6]:
arr_target= np.asarray(list(set(result_df['Target'].values)))
arr_target

array(['Financial_News_1', 'Tweets_hatred_5', 'Movie_2',
       'Financial_News_2', 'Tweets_2', 'Tweets_1', 'Movie_1'],
      dtype='<U16')

In [7]:
df_target_dic={}
for d in datasets:
    f_name = re.search('input/(.*).csv', d).group(1).split('/')[1]
    df_target_dic[f_name]=d
df_target_dic

{'Movie_1': '../input/imbd-data-1/Movie_1.csv',
 'Movie_2': '../input/imbd-data-1/Movie_2.csv',
 'Financial_News_2': '../input/financial-news-data-2/Financial_News_2.csv',
 'Financial_News_1': '../input/financial-news-data-2/Financial_News_1.csv',
 'Tweets_1': '../input/tweets-data-4/Tweets_1.csv',
 'Tweets_2': '../input/tweets-data-4/Tweets_2.csv',
 'Tweets_hatred_5': '../input/tweets-hatred-data-5/Tweets_hatred_5.csv'}

In [8]:
arr_model_already_trained=['../input/ver-1-of-model-search-dataset/Financial_News_1_trained_model',
                           '../input/ver-1-of-model-search-dataset/Financial_News_2_trained_model',
                           '../input/ver-1-of-model-search-dataset/Movie_1_trained_model',
                           '../input/ver-1-of-model-search-dataset/Movie_2_trained_model',
                           '../input/ver-1-of-model-search-dataset/Tweets_1_trained_model',
                           '../input/ver-1-of-model-search-dataset/Tweets_2_trained_model',
                           '../input/ver-1-of-model-search-dataset/Tweets_hatred_5_trained_model'
                          ]

In [9]:
a=[0.9,0.2,0.5,0.6]

def sum_threshold(x):
    if x>=2:
        return 1
    else:
        return 0

def func_threshold(x):
    if x>=0.5:
        return 1
    else:
        return 0
output_list = [func_threshold(i) for i in a]
output_list

[1, 0, 1, 1]

In [10]:
from collections import defaultdict

array1 = [1,2,3,4]  
array2 = [6,3,2,3]
array3 = [5,7,11,15]

my_dict = defaultdict(list)
my_dict[1].append(array1)
my_dict[1].append(array2)

my_dict

defaultdict(list, {1: [[1, 2, 3, 4], [6, 3, 2, 3]]})

In [11]:
metrics=['JSD','Adaptivity','L2D','Source Accuracy']
metric_dic={}
for m in metrics:
    temp_df=[]
    print("--------")
    target_dict_max_voting={}
    for target in arr_target:
        print("---------------------------------------------------------------------")
        temp_df = result_df[result_df['Target']==target][['Source','Target',m]]
        
#         if m=='Adaptivity' or m=='Source Accuracy':
#             temp_df= temp_df.nlargest(3, m)
#         else:
#             temp_df= temp_df.nsmallest(3, m)
        print("TARGET-",target)
#         print("Data-")
#         print(temp_df)
        dict_predict_curr={}
        for src in temp_df['Source'].values:
            # Model Save
            file_name = '../input/ver-1-of-model-search-dataset/'+src+"_trained_model"
#             print(file_name)
            if file_name in arr_model_already_trained: # model is already saved
                model_src = build_model(bert_model, max_len=400)
                model_src.load_weights(file_name)
                print("")
                print("Weights Loaded-      ",file_name)
            print("Target dataset loaded-    ",target)
            
            df_target=pd.read_csv(df_target_dic[target])
            df_target['text']=df_target['text'].apply(denoise_text)
            x_train_target= df_target.text
            x_train_target = fast_encode(x_train_target.values, fast_tokenizer, maxlen=400)
            y_pred = model_src.predict(x_train_target) 
            output_pred = [func_threshold(i) for i in y_pred]
            dict_predict_curr[src]=output_pred
        
#         print(dict_predict_curr)
        sum_arr=[]
        for p in range(len(y_pred)):
            s=0
            for x in dict_predict_curr:
                s+=dict_predict_curr[x][p]
            sum_arr.append(s)    
        max_voted_arr = [sum_threshold(i) for i in sum_arr]
#         print("vote freq arr-", max_voted_arr)
        max_voted_model={}
        for source_model in dict_predict_curr:
            print(source_model,"-->",target)
            for i,val in enumerate(dict_predict_curr[source_model]):
                if max_voted_arr[i]==val:
                    if source_model in max_voted_model:
                        max_voted_model[source_model]+=1
                    else:
                        max_voted_model[source_model]=1
#             print(len(output_pred))
            max_voted_model[source_model]=max_voted_model[source_model]/len(output_pred)
            
        print(max_voted_model)
        
        ####
        # creating target dict
        target_dict_max_voting[target]=max_voted_model
    print(target_dict_max_voting)
    metric_dic[m]=target_dict_max_voting
print("----##----")
print(metric_dic)    

--------
---------------------------------------------------------------------
TARGET- Financial_News_1


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



Weights Loaded-       ../input/ver-1-of-model-search-dataset/Tweets_2_trained_model
Target dataset loaded-     Financial_News_1


2022-04-13 17:19:50.441779: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)



Weights Loaded-       ../input/ver-1-of-model-search-dataset/Tweets_1_trained_model
Target dataset loaded-     Financial_News_1

Weights Loaded-       ../input/ver-1-of-model-search-dataset/Financial_News_2_trained_model
Target dataset loaded-     Financial_News_1

Weights Loaded-       ../input/ver-1-of-model-search-dataset/Movie_1_trained_model
Target dataset loaded-     Financial_News_1

Weights Loaded-       ../input/ver-1-of-model-search-dataset/Movie_2_trained_model
Target dataset loaded-     Financial_News_1

Weights Loaded-       ../input/ver-1-of-model-search-dataset/Tweets_hatred_5_trained_model
Target dataset loaded-     Financial_News_1
Tweets_2 --> Financial_News_1
Tweets_1 --> Financial_News_1
Financial_News_2 --> Financial_News_1
Movie_1 --> Financial_News_1
Movie_2 --> Financial_News_1
Tweets_hatred_5 --> Financial_News_1
{'Tweets_2': 0.8331637843336724, 'Tweets_1': 0.8036622583926755, 'Financial_News_2': 0.8677517802644964, 'Movie_1': 0.9440488301119023, 'Movie_2': 0.

In [12]:
import json
    
with open("Max_Voting_Sentiment.json", "w") as outfile:
    json.dump(metric_dic, outfile)